# Pre processing process
#### Sources adds in the page title a string to identify the provenance of the item. Since we don't want to consider the source in the clustering, we preprocess the data to remove that string from the page title before the embedding.

In [29]:
import preprocessing

In [32]:
preprocessing.process_sources(preprocessing.get_dataset_dir())

AttributeError: module 'preprocessing' has no attribute 'get_dataset_dir'